# שיעור – ניתוח אמורטי (Amortized Analysis)
מחברת זו משלבת **הסבר בעברית** + **קוד רץ** המדגים Aggregate/Accounting ויישומים נפוצים.

In [ ]:
import numpy as np, math
import matplotlib.pyplot as plt
np.random.seed(42)
print('מוכן להרצה ✅')

## 1) הרעיון האמורטי
מודדִים עלות ממוצעת לאורך m פעולות: מחשבים עלות כוללת $T(m)$, מציבים חסם $C(m)$, ואז העלות האמורטית היא $C(m)/m$.

**שיטות**: Aggregate / Accounting / Potential.

## 2) Binary Counter — Aggregate
מספר היפוכי הביטים הכולל עד $m$ אינקרמנטים קטן מ- $2m$ ⇒ העלות האמורטית לפעולה היא **O(1)**.

In [ ]:
def total_flips(m: int) -> int:
    total = 0
    i = 1
    while i <= m:
        total += m // i
        i <<= 1
    return total

m = 1024
total = total_flips(m)
print('סה"כ היפוכים:', total, '| אמורטי ≈', round(total/m,3))
xs = np.arange(1, m+1)
bound = 2*xs
flips = [total_flips(k) for k in xs]
plt.figure(); plt.plot(xs, bound, label='2m (חסם)'); plt.plot(xs, flips, label='סה"כ היפוכים')
plt.xlabel('m'); plt.ylabel('סה"כ היפוכים'); plt.title('Binary Counter: total flips ≤ 2m'); plt.legend(); plt.show()

## 3) Dynamic Array — append + resize×2
למרות ש־resize יקר, העלות הכוללת עד $m$ הוספות היא $O(m)$ ⇒ אמורטית $O(1)$.

In [ ]:
def simulate_dynamic_array_appends(m=1500, growth=2):
    cap, size = 1, 0
    total = 0
    costs_all_ops = []
    cum_after_each_append = []
    for _ in range(m):
        if size == cap:
            total += size
            costs_all_ops.append(size)
            cap *= growth
        total += 1
        costs_all_ops.append(1)
        size += 1
        cum_after_each_append.append(total)
    return np.array(costs_all_ops), np.array(cum_after_each_append), total

m = 1500
costs_all, cum_after_each, total = simulate_dynamic_array_appends(m)
print('עלות כוללת:', total, '| אמורטי ≈', round(total/m,3))
# גרף 1: כל הפעולות
plt.figure(); plt.plot(np.arange(1, len(costs_all)+1), np.cumsum(costs_all))
plt.xlabel('מספר פעולות בפועל (resize + append)'); plt.ylabel('עלות מצטברת'); plt.title('Dynamic Array: כל הפעולות'); plt.show()
# גרף 2: לאחר כל append בלבד (אורך = m)
plt.figure(); plt.plot(np.arange(1, m+1), cum_after_each)
plt.xlabel('מספר הוספות (m)'); plt.ylabel('עלות מצטברת לאחר כל append'); plt.title('Dynamic Array: עלות מצטברת לפי הוספות'); plt.show()

## 4) Clearable Table — Lazy clear (Accounting רעיוני)
לא מאפסים N תאים פיזית. שומרים חותמת גלובלית לכל clear וחותמת לכל תא. get מחזיר 0 אם התא לא עודכן מאז ה-clear האחרון ⇒ clear ב־O(1).

In [ ]:
class LazyClear:
    def __init__(self, n):
        self.n = n
        self.values = [0]*n
        self.stamp  = [0]*n
        self.cur = 1
    def clear(self):
        self.cur += 1
    def set(self, i, x):
        self.values[i] = x; self.stamp[i] = self.cur
    def get(self, i):
        return self.values[i] if self.stamp[i]==self.cur else 0

t = LazyClear(8)
t.set(0,7); t.set(3,5)
print('לפני clear:', [t.get(i) for i in range(4)])
t.clear()
print('אחרי clear:', [t.get(i) for i in range(4)])
t.set(1,9)
print('לאחר set(1,9):', [t.get(i) for i in range(4)])